In [138]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold,TimeSeriesSplit
import lightgbm as lgb
import ta
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, classification_report
import vectorbt as vbt

In [100]:
BTC = pd.read_hdf(f'Data/BTCUSDT.h5', key='my_data')
ETH = pd.read_hdf(f'Data/ETHUSDT.h5', key='my_data')


In [101]:
BTC['Date'] = pd.to_datetime(BTC['Date'])
ETH['Date'] = pd.to_datetime(ETH['Date'])
BTC = BTC.rename(columns={'Open time': 'Date', 'Close': 'BTC'})
ETH = ETH.rename(columns={'Open time': 'Date', 'Close': 'ETH'})

In [122]:
merged_data=pd.merge(BTC,ETH,on='Date',how='left').dropna()
merged_data=merged_data.set_index('Date')

In [123]:
merged_data['BTC']=merged_data['BTC'].astype(float)
merged_data['ETH']=merged_data['ETH'].astype(float)


In [124]:
def Technical_indicators(Merged,target):
    for i in Merged:
        Merged['Log_return_'+i]=np.log(Merged[i])/np.log(Merged[i].shift(5))-1

        
    window_4_hr=60*4
    window_30_M=30
    window_1_day=60*24
    window_2_day=60*24*2
    window_2_weeks=60*24*14

    Merged['RSI 30M'] = ta.momentum.RSIIndicator(Merged[target], window=window_30_M).rsi()
    Merged['RSI 30M_EMA'] = Merged['RSI 30M'].ewm(span=20, adjust=False).mean()
    Merged['RSI 4hr'] = ta.momentum.RSIIndicator(Merged[target], window=window_4_hr).rsi()
    Merged['RSI 1D'] = ta.momentum.RSIIndicator(Merged[target], window=window_1_day).rsi()
    Merged['RSI 2D'] = ta.momentum.RSIIndicator(Merged[target], window=window_2_day).rsi()
    Merged['RSI_diff'] = (Merged['RSI 2D']-Merged['RSI 1D'])
    Merged['RSI 1D_diff'] =Merged['RSI 1D'].diff()
    Merged['RSI 2Weeks'] = ta.momentum.RSIIndicator(Merged[target], window=window_2_weeks).rsi()
    Merged['ema 4HR'] = Merged[target].ewm(span=window_4_hr, adjust=False).mean()
    Merged['ema 1D'] = Merged[target].ewm(span=window_1_day, adjust=False).mean()
    Merged['ema 2W'] = Merged[target].ewm(span=window_2_weeks, adjust=False).mean()
    Merged['SMA_50'] = ta.trend.sma_indicator(Merged[target], window=50)
    Merged['SMA_200'] = ta.trend.sma_indicator(Merged[target], window=200)
    Merged['SMA_12HR'] = ta.trend.sma_indicator(Merged[target], window=60*12)
    Merged['RSI'] = ta.momentum.RSIIndicator(Merged[target], window=30).rsi()
    Merged['MACD'] = ta.trend.MACD(Merged[target], window_fast=window_30_M, window_slow=60).macd()
    Merged['MACD30M'] = Merged['MACD'].ewm(span=24*60, adjust=False).mean()
    Merged['bb_upper'] = ta.volatility.BollingerBands(Merged[target], window=60).bollinger_hband()
    Merged['bb_lower'] = ta.volatility.BollingerBands(Merged[target], window=60).bollinger_lband()
    Merged['SMA100'] = Merged[target].rolling(window=100).mean()
    rolling_stats = Merged[target].rolling(window=20).agg(['mean', 'std'])
    Merged['Middle_Band'] = rolling_stats['mean']
    Merged['Upper_Band'] = rolling_stats['mean'] + 2 * rolling_stats['std']
    Merged['Lower_Band'] = rolling_stats['mean'] - 2 * rolling_stats['std']
    Merged['+DM'] = np.where((Merged[target].diff() > 0), Merged[target].diff(), 0)
    Merged['-DM'] = np.where((Merged[target].diff() < 0), abs(Merged[target].diff()), 0)
    Merged['TR'] = np.maximum((Merged[target] - Merged[target].shift()), Merged[target].diff())
    Merged['ATR'] = Merged['TR'].rolling(window=60).mean()
    Merged['+DI'] = 100 * (Merged['+DM']/Merged['ATR']).rolling(window=60).mean()
    Merged['-DI'] = 100 * (Merged['-DM']/Merged['ATR']).rolling(window=60).mean()
    Merged['DX'] = (abs(Merged['+DI'] - Merged['-DI']) / abs(Merged['+DI'] + Merged['-DI'])) * 100
    Merged['ADX'] = Merged['DX'].rolling(window=60).mean()

    return Merged

In [125]:
target='BTC'
merged_data=Technical_indicators(merged_data,target)
merged_data['Objective']=np.where(merged_data[target].shift(-30)-merged_data[target]>100,1,0)
merged_data=merged_data.dropna()

In [126]:
merged_data

,BTC,ETH,Log_return_BTC,Log_return_ETH,RSI 30M,RSI 30M_EMA,RSI 4hr,RSI 1D,RSI 2D,RSI_diff,...,Lower_Band,+DM,-DM,TR,ATR,+DI,-DI,DX,ADX,Objective
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-14 23:59:00,39144.50,1231.62,-0.000191,0.001409,54.228581,57.978643,50.255693,51.217069,50.711155,-0.505914,...,38903.768683,0.00,167.28,-167.28,2.598000,499.849810,668.579857,14.440753,17.331823,1
2021-01-15 00:00:00,39207.17,1236.95,-0.000103,0.001903,56.378185,57.826219,50.540057,51.259075,50.729835,-0.529240,...,38922.582906,62.67,0.00,62.67,4.428000,523.438338,649.030330,10.711757,17.037565,1
2021-01-15 00:01:00,39031.95,1226.15,-0.000537,0.000697,49.635898,57.046188,49.741714,51.135880,50.676120,-0.459760,...,38927.630942,0.00,175.22,-175.22,0.253167,501.533327,1802.552384,56.465740,17.557323,1
2021-01-15 00:02:00,39184.01,1233.49,-0.000320,0.001464,54.518847,56.805489,50.424161,51.237655,50.721417,-0.516238,...,38955.890108,152.06,0.00,152.06,2.124833,610.604277,1802.552384,49.393731,17.983271,1
2021-01-15 00:03:00,39360.01,1239.87,0.000116,0.001467,59.249441,57.038246,51.194428,51.355007,50.773761,-0.581246,...,38976.121859,176.00,0.00,176.00,4.939667,668.112995,1802.552384,45.916351,18.362140,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-12 12:17:00,61108.55,2928.97,-0.000022,0.000098,46.021718,46.975259,50.862130,49.847016,49.648662,-0.198353,...,61076.432444,0.00,40.41,-40.41,-1.647000,314.225201,239.702051,13.453599,210.368546,0
2024-05-12 12:18:00,61117.63,2928.83,0.000001,0.000101,47.143553,46.991287,50.989426,49.864208,49.655776,-0.208432,...,61075.271234,9.08,0.00,9.08,-1.039667,299.669254,431.971903,18.082997,210.350168,0
2024-05-12 12:19:00,61116.61,2928.30,0.000021,0.000086,47.029968,46.994971,50.974530,49.862287,49.654988,-0.207299,...,61074.178338,0.00,1.02,-1.02,-1.115000,313.776468,430.447239,15.676841,210.341724,0


In [127]:
test=merged_data.iloc[:int(len(merged_data)*0.7)]
train=merged_data.iloc[int(len(merged_data)*0.7):]

In [128]:
features = np.array(test.loc[:, ~test.columns.isin(['Objective'])])
target = np.array(test['Objective'])

In [129]:
tscv = TimeSeriesSplit(n_splits=5)
for train_index, test_index in tscv.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]

    # Create dataset for lightgbm
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

    # Specify your configurations as a dict
    params = {
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': ['auc', 'binary_logloss'],
    'num_leaves': 40,  # reduced
    'max_depth': 5,   # reduced
    'min_data_in_leaf': 100,  # reduced
    'learning_rate': 0.05,  # increased for faster convergence
    'feature_fraction': 0.6,  # reduced to lower overfitting
    'bagging_fraction': 0.7,  # reduced
    'scale_pos_weight': 5,  # adjusted based on actual data imbalance
    'bagging_freq': 5,
    'verbose': -1
}

    # Train the model
    gbm = lgb.train(params,
                    lgb_train,
                    num_boost_round=1000,
                    valid_sets=lgb_eval, callbacks=[lgb.early_stopping(stopping_rounds=500)])
    y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)
    results.append(y_pred)

Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.554021	valid_0's binary_logloss: 0.601908
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.555379	valid_0's binary_logloss: 0.584824
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.578889	valid_0's binary_logloss: 0.505765
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[1]	valid_0's auc: 0.587934	valid_0's binary_logloss: 0.365778
Training until validation scores don't improve for 500 rounds
Early stopping, best iteration is:
[76]	valid_0's auc: 0.713912	valid_0's binary_logloss: 0.395912


In [130]:

# Convert probabilities to binary predictions with 0.5 as the threshold
y_pred_binary = [1 if prob > 0.5 else 0 for prob in y_pred]

# Evaluate model
print(f"AUC Score: {roc_auc_score(y_test, y_pred)}")
print(f"Accuracy Score: {accuracy_score(y_test, y_pred_binary)}")
print(f"F1 Score: {f1_score(y_test, y_pred_binary)}")
print(classification_report(y_test, y_pred_binary))


AUC Score: 0.7139117389429801
Accuracy Score: 0.9145778856000875
F1 Score: 0.18366668250273172
              precision    recall  f1-score   support

           0       0.94      0.97      0.95    187631
           1       0.26      0.14      0.18     13523

    accuracy                           0.91    201154
   macro avg       0.60      0.56      0.57    201154
weighted avg       0.89      0.91      0.90    201154



In [131]:
####### Test


In [133]:
new_predictions_proba = gbm.predict(train.loc[:, ~train.columns.isin(['Objective'])], num_iteration=gbm.best_iteration)

# Assuming probabilities above 0.5 indicate price will go up, otherwise down
new_predictions_class = [1 if p > 0.5 else 0 for p in new_predictions_proba]

In [137]:
train['Signal']=new_predictions_class

C:\Users\adnan\AppData\Local\Temp\ipykernel_26676\2035319227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['Signal']=new_predictions_class


In [164]:
buy_signals = train['Signal']==1

# Create the portfolio
pf = vbt.Portfolio.from_signals(train['BTC'],
                                       entries=buy_signals,
                                       # exits=exit,
                                       tp_stop=0.3,
                                       sl_stop=0.3,
                                       sl_trail=True,
                                    #    short_entries=sell_signals,
                                    #    short_exits=sell_exit,
                                       fees=0.005,init_cash=10000)
# Analyze the results
portfolio_perf = pf.stats()
print(portfolio_perf)

Start                         2023-05-10 13:31:00
End                           2024-05-12 12:21:00
Period                                     517255
Start Value                               10000.0
End Value                             21436.50468
Total Return [%]                       114.365047
Benchmark Return [%]                   117.074347
Max Gross Exposure [%]                      100.0
Total Fees Paid                        559.523704
Max Drawdown [%]                        23.288911
Max Drawdown Duration                    140998.0
Total Trades                                    4
Total Closed Trades                             3
Total Open Trades                               1
Open Trade PnL                          -7.581196
Win Rate [%]                                100.0
Best Trade [%]                          29.552358
Worst Trade [%]                         28.870053
Avg Winning Trade [%]                   29.099472
Avg Losing Trade [%]                          NaN


In [165]:
pf.plot().show()